In [1]:
%matplotlib inline

In [2]:
# global libraries & presets
import warnings
warnings.filterwarnings('ignore')

import sys, os
import numpy as np
import pandas as pd
import string
import re
from pprint import pprint
from collections import Counter

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

# Frame the problem and look at the big picture

# Get the data

In [3]:
import scrapy
from bs4 import BeautifulSoup

# pull in all csvs in working directory
all_files = os.listdir("./")    
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
all_dfs = {}

# save them all to dict
for i, file in enumerate(csv_files):
    all_dfs[i] = {'name' : file,
                  'data' : pd.read_csv(file)}
    print("File "+ str(i) + ": " + file)

# grab the data you want with all_df[i]['data'] ...
input_df = all_dfs[0]['data']


File 0: bitdefender_vpn_customer_responses.csv
File 1: bitdefender_vpn_customer_responses-extra.csv


# Explore the data

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
sns.set(rc={'figure.figsize':(10, 6)})

# save long col names to a code table
def create_varcode_table(input_df):
    colname_map = {}
    for i, colname in enumerate(input_df.columns):
        colname_map['VAR_'+str(i)] = colname

    # substitute colnames to make easier to handle
    new_cols = [ 'VAR_'+str(i) for i, col in enumerate(input_df.columns) ]  
    input_df.columns = new_cols
    
    return colname_map, input_df

colnames, df = create_varcode_table(input_df)


##### 1. Check for empty & zero variance columns and remove them

# check each variable for variation
def inspect_var_levels(df):
    var = []
    levels = []
    for col in df:
        var.append(col)
        levels.append(df[col].value_counts().count())

    col_counts = pd.DataFrame({"variable": var, "levels": levels})
    return col_counts.sort_values(by="levels")

# function to reassign dtypes
def set_dtypes(df, dtypes):
    for var in dtypes:
        df[var] = df[var].astype(dtypes[var])
    return df

# view variation of each variable
inspect_var_levels(df)

# assess values by eye
df.head(25)

# list of cols to drop
cols_to_drop = ["VAR_0", "VAR_1", "VAR_4", "VAR_8", "VAR_9"]

# remove vars with < 2 levels
df = df.drop(cols_to_drop, axis=1)

##### 2. Check for high-null value cols

#df.info()

##### 3. assign correct dtypes
dtypes = {
    'VAR_2': 'datetime64',
    'VAR_3': 'category',
    'VAR_5': 'category',
    'VAR_6': 'category', 
    'VAR_7': 'category'}

df = set_dtypes(df, dtypes)

# check variation of data

df.describe(include="category")

,VAR_3,VAR_5,VAR_6,VAR_7
count,4088,4088,4088,4088
unique,53,3,103,60
top,United States,desktop,Chrome 96.0.4664,Windows 10
freq,2043,2732,966,2115


In [16]:
##### 4. Identify combined-type object variables (aka half open-text, half-category)

def check_for_mixed_type_cols(df):
    for col in df.select_dtypes("object"):
        print("Variable: ", col)
        print(df[col].value_counts())
        print(" ")
    
#check_for_mixed_type_cols(df)
    
mixed_vars = df[["VAR_10", "VAR_14", "VAR_19", "VAR_20"]].set_index(
text_only_vars = df.select_dtypes("object").drop(mixed_vars, axis=1)

In [115]:
###### 5. Process mixed type categories (need to make a code table for each)

### for each mixed type var, assign separator string
#check_for_mixed_type_cols(mixed_vars)

### separate & cluster checkbox options first, then save into original df with 'category' dtype assigned
sep1_ = " \| "
split_ = (mixed_vars["VAR_14"]
          .str.split(sep1_, expand = True)
          .fillna(value=np.nan).reset_index()
          .rename(columns={"index":"id"}))

### melt it down while preserving the id
melted_ = split_.melt(["id"]).sort_values(by="id")
melted_ = (melted_[melted_["value"].notnull()]
           .drop("variable", axis=1))

### split again for the open-text separator
sep2_ = " \(click here to tyoe\) - "
split2_ = melted_['value'].str.split(sep2_, expand = True).fillna(value=np.nan)
melted_["value"] = split2_[0] 
melted_["value_2"] = split2_[1] 

In [116]:
melted_

,id,value,value_2
0,0,None - Bitdefender was my first choice,NaN
1,1,None - Bitdefender was my first choice,NaN
2,2,None - Bitdefender was my first choice,NaN
3,3,None - Bitdefender was my first choice,NaN
4092,4,Express VPN,NaN
...,...,...,...
8170,4082,Express VPN,NaN
12258,4082,Cyberghost VPN,NaN
4082,4082,Nord VPN,NaN
4085,4085,None - Bitdefender was my first choice,NaN


,id,value,value_2
1,1,None - Bitdefender was my first choice,NaN
2,2,None - Bitdefender was my first choice,NaN
3,3,None - Bitdefender was my first choice,NaN
4092,4,Express VPN,NaN
4,4,Nord VPN,NaN
5,5,None - Bitdefender was my first choice,NaN
6,6,None - Bitdefender was my first choice,NaN
7,7,None - Bitdefender was my first choice,NaN
8,8,None - Bitdefender was my first choice,NaN
9,9,None - Bitdefender was my first choice,NaN


# Prepare the data (transform, feature selection, etc.)

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

# Explore & shortlist models


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# gensim for topic modelling
import gensim,logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel


# Fine-tune the system (parameter tuning)

In [9]:
from sklearn.model_selection import GridSearchCV

# Present solution

# Launch